In [ ]:
import math
import torch
import gpytorch
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set up the training data (noisy sine wave)
train_x_full = torch.linspace(0, 1, 100).to(device)
train_y_full = torch.sin(train_x_full * (2 * math.pi)) + torch.randn(train_x_full.size()).to(device) * math.sqrt(0.04)

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean().to(device)
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel().to(device)
        ).to(device)
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def get_looph_predictions():
    """Get LOO predictions using LOOPH with fixed parameters"""
    # Initialize model with fixed parameters
    likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
    model = ExactGPModel(train_x_full, train_y_full, likelihood).to(device)
    
    # Set fixed parameters
    model.covar_module.base_kernel.lengthscale = 0.2
    model.covar_module.outputscale = 1.0
    likelihood.noise = 0.04
    
    # Evaluation mode
    model.eval()
    likelihood.eval()
    
    # Use LOOPH to get predictions
    with torch.no_grad():
        output = model(train_x_full)
        # Compute the full covariance matrix
        f_covar = output.covariance_matrix
        f_mean = output.mean
        
        # Get LOO predictions using the equations from RW 5.4.2
        S = f_covar + likelihood.noise * torch.eye(f_covar.shape[0]).to(device)
        S_inv = torch.linalg.inv(S)
        
        # Compute LOO means and variances
        S_inv_diag = S_inv.diag()
        loo_means = f_mean - (S_inv @ train_y_full) / S_inv_diag
        loo_vars = 1.0 / S_inv_diag
        
        # Compute negative log predictive probability
        nlpp = 0.5 * torch.log(2 * math.pi * loo_vars) + \
               0.5 * (train_y_full - loo_means).pow(2) / loo_vars
        
    return {
        'means': loo_means,
        'vars': loo_vars,
        'nlpp': nlpp.sum().item(),
        'nlpp_per_point': nlpp
    }

def get_loocv_predictions():
    """Get predictions using traditional LOOCV with fixed parameters"""
    n_points = len(train_x_full)
    predictions = []
    nlpp_values = []
    
    for left_out_idx in tqdm(range(n_points), desc="LOOCV Progress"):
        # Create training set without the left-out point
        mask = torch.ones(n_points, dtype=bool)
        mask[left_out_idx] = False
        
        train_x = train_x_full[mask].reshape(-1, 1)
        train_y = train_y_full[mask]
        test_x = train_x_full[left_out_idx].reshape(1, 1)
        test_y = train_y_full[left_out_idx]
        
        # Initialize model with fixed parameters
        likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
        model = ExactGPModel(train_x, train_y, likelihood).to(device)
        
        # Set fixed parameters
        model.covar_module.base_kernel.lengthscale = 0.2
        model.covar_module.outputscale = 1.0
        likelihood.noise = 0.04
        
        # Evaluation mode
        model.eval()
        likelihood.eval()
        
        # Predict
        with torch.no_grad():
            observed_pred = likelihood(model(test_x))
            pred_mean = observed_pred.mean
            pred_var = observed_pred.variance
            
            # Compute NLPP for this point
            nlpp = 0.5 * torch.log(2 * math.pi * pred_var) + \
                   0.5 * (test_y - pred_mean).pow(2) / pred_var
            
        predictions.append({
            'mean': pred_mean.cpu().item(),
            'variance': pred_var.cpu().item()
        })
        nlpp_values.append(nlpp.cpu().item())
    
    # Organize results
    means = np.array([p['mean'] for p in predictions])
    vars = np.array([p['variance'] for p in predictions])
    
    return {
        'means': means,
        'vars': vars,
        'nlpp': sum(nlpp_values),
        'nlpp_per_point': np.array(nlpp_values)
    }

# Get predictions from both methods
print("Computing LOOPH predictions...")
looph_results = get_looph_predictions()

print("Computing LOOCV predictions...")
loocv_results = get_loocv_predictions()

# Convert results to numpy for plotting
x_values = train_x_full.cpu().numpy()
actual_values = train_y_full.cpu().numpy()
looph_means = looph_results['means'].cpu().numpy()
looph_stds = np.sqrt(looph_results['vars'].cpu().numpy())
loocv_means = loocv_results['means']
loocv_stds = np.sqrt(loocv_results['vars'])

# Create comparison plots
plt.figure(figsize=(15, 10))

# Predictions comparison
plt.subplot(2, 1, 1)
plt.plot(x_values, actual_values, 'k*', label='Actual', markersize=4)
plt.plot(x_values, looph_means, 'b-', label='LOOPH Predictions')
plt.fill_between(x_values, 
                 looph_means - 2*looph_stds,
                 looph_means + 2*looph_stds,
                 color='blue', alpha=0.2, label='LOOPH 95% CI')
plt.plot(x_values, loocv_means, 'r--', label='LOOCV Predictions')
plt.fill_between(x_values,
                 loocv_means - 2*loocv_stds,
                 loocv_means + 2*loocv_stds,
                 color='red', alpha=0.2, label='LOOCV 95% CI')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Predictions Comparison: LOOPH vs LOOCV')
plt.legend()

# NLPP comparison
plt.subplot(2, 1, 2)
plt.plot(x_values, looph_results['nlpp_per_point'].cpu().numpy(), 'b-', label='LOOPH NLPP')
plt.plot(x_values, loocv_results['nlpp_per_point'], 'r--', label='LOOCV NLPP')
plt.xlabel('x')
plt.ylabel('NLPP')
plt.title('Negative Log Predictive Probability Comparison')
plt.legend()

plt.tight_layout()
plt.show()

# Compute and print metrics
rmse_looph = np.sqrt(np.mean((looph_means - actual_values) ** 2))
rmse_loocv = np.sqrt(np.mean((loocv_means - actual_values) ** 2))

print("\nPerformance Metrics:")
print(f"LOOPH RMSE: {rmse_looph:.4f}")
print(f"LOOCV RMSE: {rmse_loocv:.4f}")
print(f"LOOPH Total NLPP: {looph_results['nlpp']:.4f}")
print(f"LOOCV Total NLPP: {loocv_results['nlpp']:.4f}")

# Compute correlation between predictions
correlation = np.corrcoef(looph_means, loocv_means)[0,1]
print(f"\nCorrelation between LOOPH and LOOCV predictions: {correlation:.4f}")

# Plot prediction differences
plt.figure(figsize=(10, 5))
prediction_diff = looph_means - loocv_means
plt.plot(x_values, prediction_diff, 'g-', label='LOOPH - LOOCV')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
plt.xlabel('x')
plt.ylabel('Prediction Difference')
plt.title('Difference between LOOPH and LOOCV Predictions')
plt.legend()
plt.show()